# iCatcher annotations --> looking time pipeline

This script, and the helper classes that it calls, are based on Raz et al., 2024 (https://osf.io/ndkt6/), and organizes iCatcher annotations and other subject and trial level data into a format that can feed into our R looking time pipeline. It does this by organizing subject, trial, and look level data in the same format as it is usually outputed in by Datavyu manual coding. 

## Inputs: 

### 1. Lookit study-level JSON file: 

We parse the Lookit study-wide .json log (that is stored at /data/metadata/lookit_study.json), to obtain trial onsets/offsets, trial data, subject data and session data, all in one go.
    
### 2. iCatcher annotation file/s (.npz file per subject, per session)

These are the main outputs from running iCatcher. Expects one file per trial per subject, per session, named accordingly. 

### 3. raw video files (or video-parsed .json files)

needed to extract frame rates for conversion from frame rates to ms (to get look events onset/offset relative to start of video)
    
if this conversion has already been run, .json files with the relevant information should exist in the videos directory. if not, these .json files will be written in the process of obtaining this info 
   
Since we have a separate video for each trial we are not worried about calculating the exact trial onset time. However, we do store the lag between the onset of the audio and the video recording on each trial to use in our main looking time analysis in R. The original Raz et al., 2024 code (https://osf.io/ndkt6/) provides detailed instructions on how to deal with trial onsets relative to the start of videos, both when a manual onset CSV file is required and is not. The original code also provides flexibility with subject level and experiment onset information.
    

In [1]:
%load_ext autoreload
%autoreload 2

## import libraries 

In [19]:
import os
import os.path as op
from pathlib import Path
import pandas as pd
import numpy as np

from dotenv import load_dotenv
from helperfuncs.video_framerates import get_frame_information
from helperfuncs.lookit_json_parser import get_lookit_trial_times
from helperfuncs.lookit_json_parser import clean_lookit_json

load_dotenv()

True

## Set relevant paths

In [30]:
# global directory path variables 
# to do: supporting both local and server paths
project_dir = os.environ.get("PROJECT_PATH")
server_dir = os.environ.get("SERVER_PATH")

if server_dir is None:
    server_dir = project_dir

server_data_dir = op.join(server_dir, 'data')
data_dir = op.join(project_dir, 'data')

# where are icatcher outputs
icatcher_outputs_dir = op.join(server_data_dir, 'icatcher_annotations')

# where are trial info files 
trial_info_dir = op.join(data_dir, 'metadata')

# where are lookit files 
input_lookit_json = op.join(trial_info_dir, 'input_lookit_study.json')
lookit_json = op.join(trial_info_dir, 'lookit_study.json')

lookit_trial_info_csv = op.join(trial_info_dir, 'lookit_trial_timing_info.csv')

# where are subject info files 
subject_info_dir = op.join(data_dir, 'subject_info')

# where are videos 
videos_dir = op.join(server_data_dir, 'original_videos', 'mp4') 

# Making sure to clean Lookit JSON file before running this script
_ = clean_lookit_json(input_lookit_json, lookit_json)

### Create all necessary functions

#### get all non-hidden files in dir (helper function):

In [5]:
# list all files except those beginning with '.' i.e., hidden files 
def listdir_nohidden(path):
    for root, dirs, files in os.walk(path):  # Walk through each directory and subdirectory
        for f in files:
            if not f.startswith('.') and f.endswith('.npz'):  # Check conditions
                yield os.path.relpath(os.path.join(root, f), start=path) 

## Set parameters for manual error exclusions


In [25]:
include_manual_edits = 0

if include_manual_edits:
    manually_coded_sections = pd.read_csv("manual_coding_timestamps.csv")

#### convert iCatcher annotated look-events from frame-wise to timing (ms from video onset) 

In [7]:
def subsample_data(data, target_length):
    """
    Subsample the input data to match the target length.
    
    Parameters:
        data (array-like): The input data to subsample.
        target_length (int): The desired length of the output data.
    
    Returns:
        array-like: Subsampled data.
    """
    factor = len(data) / target_length
    indices = np.round(np.arange(0, len(data), factor)).astype(int)[:target_length]
    return data[indices]

def read_convert_output(filename, stamps,manual_edits_df=None):
    """
    Given an npz file containing icatcher annotated frames and looks,
    converts to pandas DataFrame with another column mapping each frame
    to its time stamp in the video
    
    INPUTS: 
    filename (string): name of tabulated iCatcher output file in format
    '[CHILD_ID]/[TRIAL_ID]_[CHILD_ID].npz'
    stamps (List[int]): time stamp for each frame, where stamps[i] is the 
    time stamp at frame i (determined in function get_frame_information(), IMPORTED function)
    
    OUTPUTS: 
    rtype: DataFrame
    
    """
    npz = np.load(filename)
    df = pd.DataFrame([])
    lst = npz.files
    npz_data = npz[lst[0]]
    confidence_data = npz[lst[1]]

    # Align frame rates of iCatcher and mp4 videos, frame rates coming in are variable because of mp4 conversion
    if len(npz_data) != len(stamps):
        if len(npz_data) > len(stamps):
            npz_data = subsample_data(npz_data, len(stamps))
            confidence_data = subsample_data(confidence_data, len(stamps))
        else:
            stamps = subsample_data(stamps, len(npz_data))
    
    df['frame'] = range(1, len(npz_data) + 1)
    df['lookType_coded'] = npz_data
    # {'noface': -2, 'nobabyface': -1, 'away': 0, 'left': 1, 'right': 2}
    df['lookType_coded'] = df['lookType_coded'].clip(lower=0)
    # convert frames to ms using frame rate
    df['time_ms'] = stamps
    df['time_ms'] = df['time_ms'].astype(int)
    
    df['confidence'] = confidence_data
    
    # SET left/right/away error FRAMES based on manual indexing, from CSV  
    if include_manual_edits: 
        if len(manual_edits_df):
            for index, row in manual_edits_df.iterrows():
                onset = row['onset']
                offset = row['offset']
                df.loc[onset:offset, 'lookType_coded'] = row['value']
                df.loc[onset:offset, 'confidence'] = -1

    # split into dfs based on when the change happens
    df['group'] = df['lookType_coded'].ne(df['lookType_coded'].shift()).cumsum()
    df_grps = df.groupby('group')
    
    dfs = []
    for _, data in df_grps:
        dfs.append(data)


    looks_onoff_grouped = pd.DataFrame()
    
    for grpI in range(len(dfs)):
        indices = dfs[grpI].index.tolist()
        # first row time_ms is onset, row after last row time_ms is offset 
        onset = df.iloc[indices[0]].time_ms

        if grpI == len(dfs)-1:
            # assume dur of last frame is the mean dur of frames 
            dur = np.floor(np.mean(np.diff(df['time_ms'])))
            offset = onset + dur
        else:
            offset = df.iloc[indices[-1]+1].time_ms
        
        lookType = df.iloc[indices[0]].lookType_coded
        confidence = np.mean(df.iloc[indices].confidence)
    
        cur_lookinfo = pd.DataFrame({"Looks.ordinal": grpI,
                    "Looks.onset" : onset,
                    "Looks.offset": offset, 
                    "Looks.lookType": lookType,
                    "Looks.confidence": confidence},  index=[0])

        looks_onoff_grouped = pd.concat([looks_onoff_grouped, cur_lookinfo], ignore_index=True)


    
    return [looks_onoff_grouped, df]

In [8]:
def get_lookit_info(child_id, trial_id, session_id, trial_info_file, lookit_json):
    
    if Path(lookit_trial_info_csv).is_file(): # check whether lookit file already parsed
        df = pd.read_csv(trial_info_file)
            
    else: # otherwise, parse and save out relevant info  
        df = get_lookit_trial_times(lookit_json)
        df.to_csv(lookit_trial_info_csv)
        
    
    # get part of df from current child
    df = df[df['SubjectInfo.subjID'] == child_id] 

    # get part of df from current trial
    df = df[df['Trials.trialID'] == trial_id]

    if 'SubjectInfo.sessionNumber' in df:
        df = df[df['SubjectInfo.sessionNumber'] == session_id] 
    else:
        df['SubjectInfo.sessionNumber'] = 1

    df = df.reset_index(drop=True)

    return df

#### get trial onsets w/r/t video

In [23]:
def get_trial_sets(child_id, trial_id, session_id, trial_info_file):
    """
    Finds corresponding trial info 
    and returns a list of [onset, offset] times for each trial in 
    milliseconds, with respect to video onset

    """

    lookit_df = get_lookit_info(child_id, trial_id, session_id, trial_info_file, lookit_json)
    df = lookit_df[[col for col in lookit_df.columns if col.startswith('Trials.')]]    
    df = df.copy()
    df['Trials.ordinal'] = df.index.values.tolist()
    df_sets = df[['Trials.audio_lag_vs_video_lag']]
    df_sets = df_sets.copy()
    df_sets.loc[:, :] = df_sets.dropna()
    trial_sets = []
    for _, trial in df_sets.iterrows():
        trial_sets.append([int(trial['Trials.audio_lag_vs_video_lag'])])
    def unique(sequence):
        seen = set()
        return [x for x in sequence if not (tuple(x) in seen or seen.add(tuple(x)))]

    return unique(trial_sets), df

#### Get subject level data 

In [9]:
def get_subject_info(child_id, trial_id, session_id, trial_info_file): 
        
    lookit_df = get_lookit_info(child_id, trial_id, session_id, trial_info_file, lookit_json)
    df = lookit_df[[col for col in lookit_df.columns if col.startswith('SubjectInfo.')]]       
    if 'SubjectInfo.sessionNumber' in df:
        df = df[df['SubjectInfo.sessionNumber'] == session_id] 
    else:
        df['SubjectInfo.sessionNumber'] = 1
        
    return df

In [16]:
print(icatcher_outputs_dir)

/Volumes/vislearnlab/experiments/visvocab/icatcher_annotations


### main function: run processes 

In [26]:
for filename in listdir_nohidden(icatcher_outputs_dir):
    # Only include files that are placed in folders set up in a directory for a particular child
    if '/' in filename and ('easy' in filename or 'hard' in filename):
        # Split into child_id and trial id
        child_id, trial_id_with_extension_child_id = filename.split('/', 1)
        # Remove the .npz extension from trial id
        trial_id_with_child_id = trial_id_with_extension_child_id.rsplit('.', 1)[0]  
        trial_id = trial_id_with_child_id.split("_")[0]
    else:
        continue
    print('child: ', child_id)
    print('trial: ', trial_id)
    # Need to pull session information in the future, our current experiment only uses a single session.
    session_id = 1

    # determine trial info files 
    trial_info_file = lookit_trial_info_csv

    # get trial onsets and offsets from input file, match to iCatcher file
    trial_sets, trials_df = get_trial_sets(child_id, trial_id, session_id, trial_info_file)
    trials_df = trials_df.reset_index(drop=True)
    
    if include_manual_edits:
        manual_edits_df = manually_coded_sections[manually_coded_sections['child_id'] == child_id and manually_coded_sections['trial_id'] == trial_id]
    
    # determine video source    
    vid_path = op.join(videos_dir, child_id, f"{trial_id}_{child_id}.mp4")
    json_video_data = op.join(videos_dir, child_id, f"{trial_id}_{child_id}.json")
    # get timestamp for each frame in the video
    timestamps, length = get_frame_information(vid_path, json_video_data)
    if not timestamps:
        print('video not found for {} in {} folder'.format(child_id, videos_dir))
        continue

    # initialize df with time stamps for iCatcher file
    icatcher_path = icatcher_outputs_dir + '/' + filename
    
    if include_manual_edits:
        [looks_df_grouped, looks_df] = read_convert_output(icatcher_path, timestamps, manual_edits_df)
    else:
        [looks_df_grouped, looks_df] = read_convert_output(icatcher_path, timestamps)
    
    looks_df_grouped = looks_df_grouped[['Looks.ordinal', 'Looks.onset', 'Looks.offset', 'Looks.lookType', 'Looks.confidence']]
    looks_df_grouped = looks_df_grouped.reset_index(drop=True)
    print(looks_df)
    # make subject level dataframe
    subject_info = get_subject_info(child_id, trial_id, session_id, trial_info_file)
    subject_info = subject_info.loc[[0], :].copy()
    subject_info['SubjectInfo.subjID'] = subject_info['SubjectInfo.subjID'].astype(str)
    subject_info = subject_info.reset_index(drop=True)
    
    
    df_concat = pd.concat([pd.DataFrame(np.repeat(subject_info.values, len(looks_df_grouped), axis=0), columns=subject_info.columns), 
                           looks_df_grouped, pd.DataFrame(np.repeat(trials_df.values, len(looks_df_grouped), axis=0), columns=trials_df.columns)], axis=1)
    
    looks_df['trial_id'] = trial_id
    looks_df['child_id'] = child_id
    # set up output name, saving a single file for each session of the experiment
    data_dir = op.join(project_dir, 'data', 'data_to_analyze')
    fname_final_output = op.join(data_dir, f'{session_id}_processed_icatcher.csv')
    # Append or create files for df_concat
    if os.path.exists(fname_final_output):
        looks_df.to_csv(fname_final_output, mode='a', header=False, index=False)  # Append without header
    else:
        looks_df.to_csv(fname_final_output, index=False)  # Create new file        


child:  ZPX2dD
trial:  hard-cheese-butter
/Volumes/vislearnlab/experiments/visvocab/data/original_videos/mp4/ZPX2dD/hard-cheese-butter_ZPX2dD.mp4
     frame  lookType_coded  time_ms  confidence  group
0        1               0       52   -1.000000      1
1        2               0      117   -1.000000      1
2        3               0      180   -1.000000      1
3        4               0      245   -1.000000      1
4        5               2      316    0.747287      2
..     ...             ...      ...         ...    ...
104    105               1     6983    0.994990      9
105    106               0     7046   -1.000000     10
106    107               0     7111   -1.000000     10
107    108               0     7184   -1.000000     10
108    109               0     7247   -1.000000     10

[109 rows x 5 columns]
child:  ZPX2dD
trial:  easy-cheese-mud
/Volumes/vislearnlab/experiments/visvocab/data/original_videos/mp4/ZPX2dD/easy-cheese-mud_ZPX2dD.mp4
     frame  lookType_coded  ti